In [ ]:
#Import necessary libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

results_csv = "runs/train/exp/results.csv"

In [ ]:
# Create a Graph of YOLO Training Metrics
if os.path.exists(results_csv):
    df = pd.read_csv(results_csv)

    # Optional smoothing
    df = df.rolling(window=3, min_periods=1).mean()

    sns.set(style="whitegrid")
    fig, axs = plt.subplots(3, 1, figsize=(14, 16), sharex=True)

    # === Box Loss Plot ===
    axs[0].set_title("Box Loss: Train vs Val")
    sns.lineplot(x='epoch', y='train/box_loss', data=df, ax=axs[0], label='Train Box Loss', color='blue')
    sns.lineplot(x='epoch', y='val/box_loss', data=df, ax=axs[0], label='Val Box Loss', color='orange')

    # Underfitting end: when val loss reaches 80% of its minimum
    val_loss_min = df['val/box_loss'].min()
    underfit_end_epoch = df[df['val/box_loss'] < 0.8 * val_loss_min]['epoch'].min()

    # Overfitting start: first increase after min val loss
    min_val_idx = df['val/box_loss'].idxmin()
    overfit_start_epoch = None
    for i in range(min_val_idx + 1, len(df) - 1):
        if df.loc[i, 'val/box_loss'] < df.loc[i + 1, 'val/box_loss']:
            overfit_start_epoch = df.loc[i + 1, 'epoch']
            break

    # Fill underfitting and overfitting spans
    if pd.notna(underfit_end_epoch):
        axs[0].axvspan(0, underfit_end_epoch, color='yellow', alpha=0.2, label='Underfitting')
    if overfit_start_epoch:
        axs[0].axvspan(overfit_start_epoch, df['epoch'].max(), color='red', alpha=0.1, label='Overfitting')
        axs[0].axvline(overfit_start_epoch, color='red', linestyle='--', label='Overfitting Start')

    axs[0].set_ylabel("Box Loss")
    axs[0].legend()

    # === Distribution Focal loss & Class Loss ===
    axs[1].set_title("Distribution Focal loss and Class Loss (Train)")
    sns.lineplot(x='epoch', y='train/dfl_loss', data=df, ax=axs[1], label='Train dfl Loss', color='green')
    sns.lineplot(x='epoch', y='train/cls_loss', data=df, ax=axs[1], label='Train Cls Loss', color='red')
    axs[1].set_ylabel("Loss")
    axs[1].legend()

    # === mAP Metrics ===
    axs[2].set_title("mAP@0.5 and mAP@0.5:0.95")
    sns.lineplot(x='epoch', y='metrics/mAP50(B)', data=df, ax=axs[2], label='mAP@0.5', color='black')
    sns.lineplot(x='epoch', y='metrics/mAP50-95(B)', data=df, ax=axs[2], label='mAP@0.5:0.95', color='magenta')
    axs[2].set_xlabel("Epoch")
    axs[2].set_ylabel("mAP")
    axs[2].legend()

    # === Final Layout ===
    plt.suptitle("YOLO Training Metrics - 50 Epochs", fontsize=18)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    os.makedirs("plots", exist_ok=True)
    plt.savefig("plots/training_metrics.png")
    plt.show()

    print("Training graph saved as: plots/training_metrics.png")

else:
    print("results.csv not found — cannot plot training metrics.")
